In [ ]:
import math
from IPython.display import display, Math
from pydantic import BaseModel, Field
from fractions import Fraction


In [36]:



class polynomial(BaseModel):
    variable: str = Field(default="x", description="Variable of the polynomial")
    coefficents: list[Fraction] = Field(default=[], description="Exponents of the polynomial, where the first is the lowest order term, i.e x^0 (1)")
    coefficients_neg_exp: list[Fraction] = Field(default=[], description="Negative exponents of the polynomial, where the first is the lowest order term, i.e x^-1")
    def frac_as_str(self, fraction: Fraction):
        if  fraction.is_integer():
            return f"{int(fraction)}"
        else:
            return "\\frac{%g}{%g}" % (fraction.numerator, fraction.denominator)
    def latex_dump(self):

        terms = []

        for exponent in range(0, len(self.coefficents)):

            if self.coefficents[exponent] == 0 & exponent != 0:
                continue # There is no need to print the term, as it is 0, as long as it is not both coefficient and exponent
            elif self.coefficents[exponent] == 1 & exponent != 0:
                coefficient = "" # If exponent is not 0, and the coefficient is 1, we will not print it
            elif self.coefficents[exponent] < 0:
                coefficient = f"- {self.frac_as_str(abs(self.coefficents[exponent]))}" # Prettier way of writing -1
            else:
                coefficient = f"+ {self.frac_as_str(self.coefficents[exponent])}"


            if exponent == 0:
                var_exponent = " " # x^0 is equal to 1, so we don't print it as 1 * y is y
            elif exponent == 1:
                var_exponent = f"{self.variable} "
            else:
                var_exponent = f"{self.variable}^{exponent} "

            term = f"{coefficient}{var_exponent}"
            print(term)
            terms.insert(0, term)
        response = "".join(terms)
        if response[0] == "+":
            return response[1:] # We don't want the first +'
        return "".join(terms)
        print(terms)

display(Math(polynomial(coefficents=[-0.5,-2,3]).latex_dump()))


- \frac{1}{2} 
- 2x 
+ 3x^2 


<IPython.core.display.Math object>